## 1. Introduction
<p><img src="https://assets.datacamp.com/production/project_1197/img/google_play_store.png" alt="Google Play logo"></p>
<p>Mobile apps are everywhere. They are easy to create and can be very lucrative from the business standpoint. Specifically, Android is expanding as an operating system and has captured more than 74% of the total market<sup><a href="https://www.statista.com/statistics/272698/global-market-share-held-by-mobile-operating-systems-since-2009">[1]</a></sup>. </p>
<p>The Google Play Store apps data has enormous potential to facilitate data-driven decisions and insights for businesses. In this notebook, we will analyze the Android app market by comparing ~10k apps in Google Play across different categories. We will also use the user reviews to draw a qualitative comparision between the apps.</p>
<p>The dataset you will use here was scraped from Google Play Store in September 2018 and was published on <a href="https://www.kaggle.com/lava18/google-play-store-apps">Kaggle</a>. Here are the details: <br>
<br></p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/apps.csv</b></div>
This file contains all the details of the apps on Google Play. There are 9 features that describe a given app.
<ul>
    <li><b>App:</b> Name of the app</li>
    <li><b>Category:</b> Category of the app. Some examples are: ART_AND_DESIGN, FINANCE, COMICS, BEAUTY etc.</li>
    <li><b>Rating:</b> The current average rating (out of 5) of the app on Google Play</li>
    <li><b>Reviews:</b> Number of user reviews given on the app</li>
    <li><b>Size:</b> Size of the app in MB (megabytes)</li>
    <li><b>Installs:</b> Number of times the app was downloaded from Google Play</li>
    <li><b>Type:</b> Whether the app is paid or free</li>
    <li><b>Price:</b> Price of the app in US$</li>
    <li><b>Last Updated:</b> Date on which the app was last updated on Google Play </li>

</ul>
</div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/user_reviews.csv</b></div>
This file contains a random sample of 100 <i>[most helpful first](https://www.androidpolice.com/2019/01/21/google-play-stores-redesigned-ratings-and-reviews-section-lets-you-easily-filter-by-star-rating/)</i> user reviews for each app. The text in each review has been pre-processed and passed through a sentiment analyzer.
<ul>
    <li><b>App:</b> Name of the app on which the user review was provided. Matches the `App` column of the `apps.csv` file</li>
    <li><b>Review:</b> The pre-processed user review text</li>
    <li><b>Sentiment Category:</b> Sentiment category of the user review - Positive, Negative or Neutral</li>
    <li><b>Sentiment Score:</b> Sentiment score of the user review. It lies between [-1,1]. A higher score denotes a more positive sentiment.</li>

</ul>
</div>
<p>From here on, it will be your task to explore and manipulate the data until you are able to answer the three questions described in the instructions panel.<br></p>

In [60]:
import pandas as pd
import numpy as np

In [61]:
app_df = pd.read_csv('datasets/apps.csv')

In [62]:
app_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9659 entries, 0 to 9658
Data columns (total 9 columns):
App             9659 non-null object
Category        9659 non-null object
Rating          8196 non-null float64
Reviews         9659 non-null int64
Size            8432 non-null float64
Installs        9659 non-null object
Type            9659 non-null object
Price           9659 non-null float64
Last Updated    9659 non-null object
dtypes: float64(3), int64(1), object(5)
memory usage: 679.2+ KB


In [63]:
app_df.isna().sum()

App                0
Category           0
Rating          1463
Reviews            0
Size            1227
Installs           0
Type               0
Price              0
Last Updated       0
dtype: int64

## 1. Clean the Installscolumn to convert it into integer data type

In [64]:
def convert(x):
    x = x.replace(',', '')
    x = x.replace('+', '')
    return int(x)

In [65]:
converted_installs = app_df['Installs'].apply(convert)

#### Save to DataFrame apps

In [66]:
apps = app_df.copy()
apps['Installs'] = converted_installs

In [67]:
apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,"January 7, 2018"
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,"January 15, 2018"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,"August 1, 2018"
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,"June 8, 2018"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,"June 20, 2018"


## 2. Find the number of apps in each category, the average price, and the average rating

In [68]:
app_category_info = pd.DataFrame()

In [69]:
# Cách 1:

app_category_info['Number of apps'] = apps.groupby('Category')['App'].count()
app_category_info['Average rating'] = apps.groupby('Category')['Rating'].mean()
app_category_info['Average price'] = apps.groupby('Category')['Price'].mean()
# Output của 3 lines bên trên sẽ ra Series

app_category_info.reset_index(inplace=True)

In [70]:
# Cách 2:

x = apps.groupby('Category').agg({'App': 'count', 'Rating': 'mean', 'Price': 'mean'})
# x sẽ là DataFrame

x.rename(columns={'App': 'Number of apps', 'Rating': 'Average rating', 'Price': 'Average price'}, inplace=True)
x.reset_index(inplace=True)
x.head()

,Category,Number of apps,Average rating,Average price
0,ART_AND_DESIGN,64,4.357377,0.093281
1,AUTO_AND_VEHICLES,85,4.190411,0.158471
2,BEAUTY,53,4.278571,0.000000
3,BOOKS_AND_REFERENCE,222,4.344970,0.539505
4,BUSINESS,420,4.098479,0.417357


#### Save answer as a DataFrame app_category_info

In [71]:
app_category_info.head()

,Category,Number of apps,Average rating,Average price
0,ART_AND_DESIGN,64,4.357377,0.093281
1,AUTO_AND_VEHICLES,85,4.190411,0.158471
2,BEAUTY,53,4.278571,0.000000
3,BOOKS_AND_REFERENCE,222,4.344970,0.539505
4,BUSINESS,420,4.098479,0.417357


## 3. Find the top 10 free FINANCE apps having the highest average sentiment score

In [72]:
user_reviews_df = pd.read_csv('datasets/user_reviews.csv')
user_reviews_df.head()

,App,Review,Sentiment Category,Sentiment Score
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25
2,10 Best Foods for You,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40
4,10 Best Foods for You,Best idea us,Positive,1.00


In [73]:
user_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 4 columns):
App                   64295 non-null object
Review                37427 non-null object
Sentiment Category    37432 non-null object
Sentiment Score       37432 non-null float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


In [74]:
# Take all apps finace if it's free from table apps

finace_free_apps = apps[(apps['Category'] == 'FINANCE') & (apps['Type'] == 'Free')]
finace_free_apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated
837,K PLUS,FINANCE,4.4,124424,NaN,10000000,Free,0.0,"June 26, 2018"
838,ING Banking,FINANCE,4.4,39041,NaN,1000000,Free,0.0,"August 3, 2018"
839,Citibanamex Movil,FINANCE,3.6,52306,42.0,5000000,Free,0.0,"July 27, 2018"
840,The postal bank,FINANCE,3.7,36718,NaN,5000000,Free,0.0,"July 16, 2018"
841,KTB Netbank,FINANCE,3.8,42644,19.0,5000000,Free,0.0,"June 28, 2018"


In [79]:
# Cal mean sentiment score of each app

mean_sentiment_score = user_reviews_df.groupby('App').agg({'Sentiment Score': 'mean'})
mean_sentiment_score.dropna(inplace=True)

In [76]:
# Right merge table mean_sentiment_score with table finance_free_apps
# to take all sentiment score of finance apps

user_feedback_finance = mean_sentiment_score.merge(finace_free_apps, on='App', how='right')

#### Save answer as a DataFrame top_10_user_feedback

In [77]:
top_10_user_feedback = user_feedback_finance.sort_values('Sentiment Score', ascending=False).iloc[:10]
top_10_user_feedback.reset_index(inplace=True)

In [78]:
top_10_user_feedback

,index,App,Sentiment Score,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated
0,6,BBVA Spain,0.515086,FINANCE,4.2,36746,NaN,5000000,Free,0.0,"July 24, 2018"
1,4,Associated Credit Union Mobile,0.388093,FINANCE,4.7,3290,12.0,50000,Free,0.0,"May 30, 2018"
2,9,BankMobile Vibe App,0.353455,FINANCE,4.3,14627,23.0,1000000,Free,0.0,"July 9, 2018"
3,0,A+ Mobile,0.329592,FINANCE,3.9,730,6.3,10000,Free,0.0,"June 26, 2018"
4,24,Current debit card and app made for teens,0.327258,FINANCE,4.3,685,21.0,50000,Free,0.0,"August 3, 2018"
5,7,BZWBK24 mobile,0.326883,FINANCE,4.5,64959,39.0,1000000,Free,0.0,"June 18, 2018"
6,29,"Even - organize your money, get paid early",0.283929,FINANCE,4.8,12304,21.0,100000,Free,0.0,"August 2, 2018"
7,21,Credit Karma,0.270052,FINANCE,4.7,706301,NaN,10000000,Free,0.0,"July 31, 2018"
8,32,Fortune City - A Finance App,0.266966,FINANCE,4.6,49275,91.0,500000,Free,0.0,"July 17, 2018"
9,14,Branch,0.264230,FINANCE,4.6,69973,3.8,1000000,Free,0.0,"July 23, 2018"
